Analyser du HTML pour trouver les "top-level readable nodes".

Obtenir la hauteur de la première ligne d’un paragraphe (dans le DOM).

On va rester au plus proche du HTML pur, donc probablement avec Python (sans librairie externe) pour la première tâche, et du JavaScript pour la seconde (car seule une exécution dans un navigateur permet de calculer le line-height ou offsetHeight).

✅ 1. Python : Trouver les "Top-Level Readable Nodes" dans du HTML
Un "top-level readable node" peut être défini comme un nœud de type texte significatif (non vide) ou un élément contenant du texte lisible (ex. : <p>, <div>, <article>, etc.), qui n’est pas imbriqué dans un autre parent lisible.

In [ ]:
from html.parser import HTMLParser

class ReadableHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.readable_tags = {'p', 'div', 'article', 'section', 'main', 'span'}
        self.stack = []
        self.top_readable_nodes = []

    def handle_starttag(self, tag, attrs):
        self.stack.append({'tag': tag, 'text': '', 'children': []})

    def handle_endtag(self, tag):
        if not self.stack:
            return

        node = self.stack.pop()
        if node['tag'] == tag:
            text = node['text'].strip()
            if tag in self.readable_tags and text:
                if self.stack:
                    self.stack[-1]['children'].append(node)
                else:
                    self.top_readable_nodes.append(node)

    def handle_data(self, data):
        if self.stack:
            self.stack[-1]['text'] += data

# Exemple d'utilisation
html = """
<html>
  <body>
    <div>
      <p>Bonjour le monde</p>
    </div>
    <section>Juste un test</section>
    <footer>Pas important</footer>
  </body>
</html>
"""

parser = ReadableHTMLParser()
parser.feed(html)

for node in parser.top_readable_nodes:
    print(f"[{node['tag']}] → {node['text']}")


✅ 2. JavaScript : Obtenir la hauteur de la première ligne d’un paragraphe

Range : permet de sélectionner une portion du DOM.

getClientRects() : renvoie un tableau des rectangles correspondant aux lignes de texte.

rects[0] : correspond à la première ligne.

In [ ]:
function getFirstLineHeight(el) {
    const range = document.createRange();
    range.setStart(el.firstChild, 0);
    range.setEnd(el.firstChild, 1); // Suppose qu'on a du texte

    const rects = range.getClientRects();
    if (rects.length > 0) {
        return rects[0].height;
    } else {
        return null;
    }
}

// Exemple d'utilisation
const p = document.querySelector('p');
const firstLineHeight = getFirstLineHeight(p);
console.log('Hauteur de la première ligne :', firstLineHeight, 'px');


🧠 Stratégie :
Tu vas parser le HTML et extraire des éléments "lisibles" en Python, puis simuler ou approximer la hauteur d’une ligne de texte, puisque Python ne peut pas obtenir la hauteur réelle d’un texte rendu dans un navigateur (car il ne peut pas calculer les clientRects).

🧰 Choix :
Utiliser html.parser ou ElementTree pour parser du HTML.

Simuler une hauteur de ligne via :

Une estimation à partir du CSS inline si dispo (style="font-size:16px" → line-height ≈ 1.2 * font-size)

Sinon une valeur par défaut

In [ ]:
from html.parser import HTMLParser
import re

class ReadableNode:
    def __init__(self, tag, text, line_height=None):
        self.tag = tag
        self.text = text.strip()
        self.line_height = line_height

    def __repr__(self):
        return f"<{self.tag}> \"{self.text}\" — line-height: {self.line_height}px"

class ReadableHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.stack = []
        self.top_readable_nodes = []
        self.readable_tags = {'p', 'div', 'article', 'section', 'main', 'span'}
        self.default_font_size = 16  # px

    def handle_starttag(self, tag, attrs):
        style_dict = {}
        for attr in attrs:
            if attr[0] == 'style':
                style = attr[1]
                for rule in style.split(';'):
                    if ':' in rule:
                        key, val = rule.split(':', 1)
                        style_dict[key.strip()] = val.strip()

        font_size = self.extract_font_size(style_dict.get('font-size', ''))
        line_height = self.estimate_line_height(font_size)

        self.stack.append({'tag': tag, 'text': '', 'line_height': line_height})

    def handle_endtag(self, tag):
        if not self.stack:
            return

        node = self.stack.pop()
        if node['tag'] != tag:
            return

        text = node['text'].strip()
        if text and tag in self.readable_tags:
            if not self.stack:  # top-level
                self.top_readable_nodes.append(
                    ReadableNode(tag, text, node['line_height'])
                )

    def handle_data(self, data):
        if self.stack:
            self.stack[-1]['text'] += data

    def extract_font_size(self, font_size_str):
        match = re.match(r'(\d+)(px)?', font_size_str)
        if match:
            return int(match.group(1))
        return self.default_font_size

    def estimate_line_height(self, font_size):
        return round(font_size * 1.2)

# HTML d'exemple
html = """
<html>
  <body>
    <div style="font-size: 18px;">
      <p>Bonjour le monde</p>
    </div>
    <section style="font-size: 14px;">Juste un test</section>
    <footer>Ne compte pas</footer>
  </body>
</html>
"""

parser = ReadableHTMLParser()
parser.feed(html)

for node in parser.top_readable_nodes:
    print(node)


🎯 1. Mouse Events en Python (équivalent conceptuel)
En JavaScript :

click, mousemove, mousedown, mouseup, etc. sont des événements liés à la souris dans le navigateur.

On y associe des callbacks via addEventListener.

En Python :

Il n'y a pas de souris dans un script CLI, mais on peut simuler une architecture orientée événement, ou utiliser des bibliothèques comme tkinter ou pygame qui gèrent ça.

In [ ]:
class MouseEvent:
    def __init__(self, event_type, x, y):
        self.type = event_type  # 'click', 'move', etc.
        self.x = x
        self.y = y

class DOMElement:
    def __init__(self, name):
        self.name = name
        self.listeners = {}

    def add_event_listener(self, event_type, callback):
        self.listeners[event_type] = callback

    def dispatch_event(self, event):
        if event.type in self.listeners:
            self.listeners[event.type](event)

# Exemple d’utilisation
def on_click(event):
    print(f"Clicked at ({event.x}, {event.y}) on <div>")

div = DOMElement("div")
div.add_event_listener("click", on_click)

# Simulation d’un clic
fake_event = MouseEvent("click", 100, 200)
div.dispatch_event(fake_event)


🎯 2. DOM Traversal en Python
En JavaScript :

On traverse avec element.parentNode, element.children, nextSibling, querySelector, etc.

En Python :

On peut recréer un arbre DOM et naviguer dedans via des attributs .parent, .children, etc.

In [ ]:
class Node:
    def __init__(self, tag, parent=None):
        self.tag = tag
        self.children = []
        self.parent = parent

    def add_child(self, child_node):
        child_node.parent = self
        self.children.append(child_node)

    def traverse(self):
        print(f"<{self.tag}>")
        for child in self.children:
            child.traverse()

# Construction de l’arbre
root = Node("html")
body = Node("body")
div = Node("div")
p = Node("p")
text = Node("text")

root.add_child(body)
body.add_child(div)
div.add_child(p)
p.add_child(text)

# Traversal
print("DOM traversal:")
root.traverse()

# Navigation
print("\nParent of <p>:", p.parent.tag)
print("Children of <div>:", [child.tag for child in div.children])


✅ Question 1 : Implémenter un LRU Cache avec TTL
LRU (Least Recently Used) + TTL (Time To Live) = on doit :

Évincer l’entrée la plus ancienne et la moins utilisée récemment quand la capacité est atteinte.

Supprimer les entrées expirées (TTL dépassé).

In [ ]:
import time

class LRUCacheTTL:
    def __init__(self, capacity, ttl_seconds):
        self.capacity = capacity
        self.ttl = ttl_seconds
        self.cache = {}  # key: (value, timestamp)
        self.order = []  # keeps keys in LRU order

    def get(self, key):
        self.cleanup()
        if key in self.cache:
            value, timestamp = self.cache[key]
            if time.time() - timestamp <= self.ttl:
                self._update_order(key)
                return value
            else:
                self._remove(key)
        return None

    def put(self, key, value):
        self.cleanup()
        if key in self.cache:
            self.cache[key] = (value, time.time())
            self._update_order(key)
        else:
            if len(self.cache) >= self.capacity:
                lru = self.order.pop(0)
                self.cache.pop(lru)
            self.cache[key] = (value, time.time())
            self.order.append(key)

    def _update_order(self, key):
        if key in self.order:
            self.order.remove(key)
        self.order.append(key)

    def _remove(self, key):
        if key in self.cache:
            del self.cache[key]
        if key in self.order:
            self.order.remove(key)

    def cleanup(self):
        expired = [key for key, (_, t) in self.cache.items() if time.time() - t > self.ttl]
        for key in expired:
            self._remove(key)


✅ Question 2 : Parse an SSML string into a node tree
SSML = Speech Synthesis Markup Language (du XML avec des balises comme <speak>, <prosody>, etc.)

Objectif :
Parser une chaîne SSML et créer une structure d’arbre de nœuds (objet Node avec tag, attrs, children, etc.)

In [ ]:
from xml.parsers.expat import ParserCreate

class SSMLNode:
    def __init__(self, tag, attrs=None, parent=None):
        self.tag = tag
        self.attrs = attrs or {}
        self.children = []
        self.text = ""
        self.parent = parent

    def __repr__(self, level=0):
        indent = "  " * level
        s = f"{indent}<{self.tag} {self.attrs}>: {self.text.strip()}\n"
        for child in self.children:
            s += child.__repr__(level + 1)
        return s

class SSMLParser:
    def __init__(self):
        self.root = None
        self.current = None
        self.parser = ParserCreate()
        self.parser.StartElementHandler = self.start
        self.parser.EndElementHandler = self.end
        self.parser.CharacterDataHandler = self.data

    def parse(self, ssml_str):
        self.parser.Parse(ssml_str)
        return self.root

    def start(self, tag, attrs):
        node = SSMLNode(tag, attrs, parent=self.current)
        if self.current:
            self.current.children.append(node)
        else:
            self.root = node
        self.current = node

    def end(self, tag):
        if self.current:
            self.current = self.current.parent

    def data(self, text):
        if self.current:
            self.current.text += text


✅ Question 3 : Transformer un arbre SSML en chaîne
On inverse la logique : on reconstruit la chaîne SSML à partir de l’arbre d’objets SSMLNode.

In [ ]:
def ssml_node_to_string(node):
    attrs_str = " ".join(f"{k}='{v}'" for k, v in node.attrs.items())
    open_tag = f"<{node.tag}{(' ' + attrs_str) if attrs_str else ''}>"
    inner = node.text
    for child in node.children:
        inner += ssml_node_to_string(child)
    close_tag = f"</{node.tag}>"
    return open_tag + inner + close_tag


🎯 Exercice : Implémente un parseur et sérialiseur de balisage vocal (mini-SSML) avec cache
Contexte :
Tu dois implémenter un système de cache (LRU + TTL) pour des blocs SSML extraits d’un texte enrichi (pseudo balisage XML), que tu dois :

Parser en arbre

Sérialiser en chaîne

Mémoriser les résultats dans un cache pour accélérer les requêtes futures.

In [ ]:
# <speak>Hello <emphasis>world</emphasis>!</speak>

Node(tag='speak', children=[
    TextNode('Hello '),
    Node(tag='emphasis', children=[TextNode('world')]),
    TextNode('!')
])


💡 Étape 3 : Transformer cet arbre en une chaîne SSML (identique à l’entrée)

In [ ]:
# Exemple
parser = SSMLParser()
tree = parser.parse("<speak>Hello <emphasis>world</emphasis>!</speak>")
print(tree)

ssml_str = ssml_node_to_string(tree)
print(ssml_str)  # Doit redonner la même chaîne

cache = LRUCacheTTL(capacity=3, ttl_seconds=60)
key = "<speak>Hello <emphasis>world</emphasis>!</speak>"
cached = cache.get(key)
if cached is None:
    parsed = parser.parse(key)
    cache.put(key, parsed)


✅ Implémentation complète
1. Node et TextNode

In [ ]:
class TextNode:
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f"TextNode('{self.text}')"

class Node:
    def __init__(self, tag, attrs=None):
        self.tag = tag
        self.attrs = attrs or {}
        self.children = []

    def __repr__(self, level=0):
        indent = "  " * level
        rep = f"{indent}<{self.tag}>\n"
        for child in self.children:
            if isinstance(child, Node):
                rep += child.__repr__(level + 1)
            else:
                rep += f"{'  ' * (level+1)}{child}\n"
        rep += f"{indent}</{self.tag}>\n"
        return rep


2. Parser SSML simplifié

In [ ]:
from xml.parsers.expat import ParserCreate

class SSMLParser:
    def __init__(self):
        self.root = None
        self.current = None
        self.parser = ParserCreate()
        self.parser.StartElementHandler = self.start
        self.parser.EndElementHandler = self.end
        self.parser.CharacterDataHandler = self.text

    def parse(self, ssml_str):
        self.root = None
        self.current = None
        self.parser.Parse(ssml_str)
        return self.root

    def start(self, tag, attrs):
        node = Node(tag, attrs)
        if self.current:
            self.current.children.append(node)
        else:
            self.root = node
        self.current = node

    def end(self, tag):
        if self.current and self.current.tag == tag:
            self.current = self.find_parent(self.root, self.current)

    def text(self, data):
        if self.current and data.strip():
            self.current.children.append(TextNode(data))

    def find_parent(self, node, target):
        if target in node.children:
            return node
        for child in node.children:
            if isinstance(child, Node):
                parent = self.find_parent(child, target)
                if parent:
                    return parent
        return None


3. Re-sérialisation d’un arbre vers SSML

In [ ]:
def ssml_node_to_string(node):
    if isinstance(node, TextNode):
        return node.text
    attrs = " ".join(f"{k}='{v}'" for k, v in node.attrs.items())
    open_tag = f"<{node.tag}{(' ' + attrs) if attrs else ''}>"
    inner = ''.join(ssml_node_to_string(child) for child in node.children)
    close_tag = f"</{node.tag}>"
    return open_tag + inner + close_tag


4. Cache LRU + TTL

In [ ]:
import time

class LRUCacheTTL:
    def __init__(self, capacity, ttl_seconds):
        self.capacity = capacity
        self.ttl = ttl_seconds
        self.cache = {}  # key: (value, timestamp)
        self.order = []

    def get(self, key):
        self.cleanup()
        if key in self.cache:
            value, timestamp = self.cache[key]
            if time.time() - timestamp <= self.ttl:
                self._update_order(key)
                return value
        return None

    def put(self, key, value):
        self.cleanup()
        if key in self.cache:
            self.cache[key] = (value, time.time())
            self._update_order(key)
        else:
            if len(self.cache) >= self.capacity:
                oldest = self.order.pop(0)
                del self.cache[oldest]
            self.cache[key] = (value, time.time())
            self.order.append(key)

    def _update_order(self, key):
        if key in self.order:
            self.order.remove(key)
        self.order.append(key)

    def cleanup(self):
        expired = [k for k, (_, t) in self.cache.items() if time.time() - t > self.ttl]
        for k in expired:
            if k in self.order:
                self.order.remove(k)
            del self.cache[k]


Exemple complet (à coller à la fin) :

In [ ]:
parser = SSMLParser()
input_str = "<speak>Hello <prosody pitch='high'>world</prosody>!</speak>"

# Avec cache
cache = LRUCacheTTL(capacity=3, ttl_seconds=60)
tree = cache.get(input_str)
if tree is None:
    print("Parsing fresh input...")
    tree = parser.parse(input_str)
    cache.put(input_str, tree)
else:
    print("Using cached tree.")

print("\n=== Node Tree ===")
print(tree)

print("\n=== Back to SSML ===")
print(ssml_node_to_string(tree))
